In [1]:
!nvidia-smi

Tue Apr 26 15:52:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   33C    P3    24W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
import torch as t
env = TictactoeEnv()

In [2]:
t.cuda.is_available()

True

In [13]:
from collections import namedtuple, deque
import random

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [14]:

memory = ReplayMemory(4)
memory.push(1, 2, 3, 4)
memory.push(2, 3, 4, 5)
memory.push(3, 4, None, 6)

transitions = memory.sample(3)

In [15]:
batch = Transition(*zip(*transitions))

In [16]:
print(batch)

Transition(state=(2, 1, 3), action=(3, 2, 4), next_state=(4, 3, None), reward=(5, 4, 6))


In [34]:
batch_reward = t.tensor(batch.reward)


# non final next states :
batch_next_state = t.tensor([ns for ns in batch.next_state if ns is not None])

# indices of final states :
final_indices = t.tensor([ind for ind in range(len(batch.next_state)) if batch.next_state[ind] == None])

# final states mask :
final_states_mask = t.tensor(tuple(map(lambda s: s is None,
                                          batch.next_state)), dtype=t.bool)
non_final_states_mask = ~final_states_mask
#print(final_states_mask)
#print(non_final_states_mask)

# initialize targets tensor :
targets = t.zeros_like(final_states_mask).type(t.LongTensor)

#targets.masked_fill(non_final_states_mask, batch_next_state)
#targets.masked_fill(final_states_mask, batch_reward)

targets[non_final_states_mask] = batch_next_state
targets[final_states_mask] = batch_reward[final_indices]

print(targets)



tensor([4, 3, 6])
